In [5]:
import pandas as pd
import numpy as np

In [6]:
### This data is a synthetic data generated by GAN model
samples = pd.read_csv('synthetic-imports-declarations.csv')

In [7]:
samples

,OFFICE,CIF_USD_EQUIVALENT,TARIFF.CODE,QUANTITY,ORIGIN.CODE,GROSS.WEIGHT,TOTAL.TAXES.USD,illicit,RAISED_TAX_AMOUNT.USD
0,OFFICE76,1943.388289,8703232926,1,CNTRY376,96.305598,468.218151,0,0
1,OFFICE51,39097.784585,2710193100,16784,CNTRY653,343358.358598,94337.473500,0,0
2,OFFICE92,93224.773942,6214300000,656,CNTRY680,40675.081867,10825.920639,0,0
3,OFFICE59,5160.500757,8703222900,1,CNTRY759,24965.116673,758.079630,0,0
4,OFFICE51,81152.409190,3507900000,17655,CNTRY528,150957.306315,30518.286546,0,0
...,...,...,...,...,...,...,...,...,...
99995,OFFICE51,61176.407534,8464900000,126,CNTRY277,336806.197108,3820.754635,0,0
99996,OFFICE60,1068.969969,8703232926,1,CNTRY759,43.969601,925.896409,0,0
99997,OFFICE45,799.860297,8481900000,1,CNTRY825,32.224977,408.178992,0,0
99998,OFFICE60,887.735852,8703232926,1,CNTRY759,198.555892,61.509931,0,0


In [ ]:
### Now we change it to fit the format described in our paper.

In [8]:
samples['sgd.id'] = list(range(1,100001))
samples['sgd.id'] = 'SGD'+samples['sgd.id'].astype(str)

In [9]:
# Refer to original data to generate some auxiliary columns
df = pd.read_csv("../../../Sharedfolder/Ndata.merged.anonymized.single.relabeled.csv", encoding="ISO-8859-1")

In [10]:
df = df.sample(100000)

In [11]:
df.columns

Index(['YEAR', 'OFFICE', 'SGD.NUMBER', 'SGD.DATE', 'RECEIPT.NUMBER',
       'RECEIPT.DATE', 'IMPORTER.TIN', 'DECLARANT.CODE', 'FOB.VALUE',
       'CIF.VALUE', 'TARIFF.CODE', 'TARIFF.DESCRIPTION', 'QUANTITY',
       'TOTAL.TAXES', 'GROSS.WEIGHT', 'INSPECTION.INFORMATION',
       'SELECTIVITY.LANE', 'EXCHANGERATE', 'CIF_USD_EQUIVALENT', 'illicit',
       'ISO3', 'RAISED_TAX_AMOUNT'],
      dtype='object')

In [12]:
samples['importer.id'] = list(df['IMPORTER.TIN'])
samples['declarant.id'] = list(df['DECLARANT.CODE'])

In [13]:
samples['country'] = samples['ORIGIN.CODE']
del samples['ORIGIN.CODE']

In [14]:
samples['revenue'] = samples['RAISED_TAX_AMOUNT.USD']
del samples['RAISED_TAX_AMOUNT.USD']

In [15]:
samples['total.taxes'] = samples['TOTAL.TAXES.USD']
del samples['TOTAL.TAXES.USD']

In [17]:
samples['gross.weight'] = samples['GROSS.WEIGHT']
samples['cif.value'] = samples['CIF_USD_EQUIVALENT']

In [57]:
from numpy import random
samples['fob.value'] = samples['cif.value'].apply(lambda x: x * min(random.normal(loc=0.8, scale=0.1), 1))

In [59]:
del samples['CIF_USD_EQUIVALENT']

In [61]:
samples['tariff.code'] = samples['TARIFF.CODE']
samples['quantity'] = samples['QUANTITY']
samples['gross.weight'] = samples['GROSS.WEIGHT']
samples['office.id'] = samples['OFFICE']

del samples['TARIFF.CODE']
del samples['QUANTITY']
del samples['GROSS.WEIGHT']
del samples['OFFICE']

In [66]:
samples['sgd.date'] = list(df['SGD.DATE'].sort_values())

In [68]:
samples = samples[['sgd.id','sgd.date','importer.id','declarant.id','country','office.id','tariff.code','quantity','gross.weight','fob.value','cif.value','total.taxes','illicit','revenue']]

In [69]:
samples

,sgd.id,sgd.date,importer.id,declarant.id,country,office.id,tariff.code,quantity,gross.weight,fob.value,cif.value,total.taxes,illicit,revenue
0,SGD1,13-01-02,IMP56598,DEC575763,CNTRY376,OFFICE76,8703232926,1,96.305598,1530.677349,1943.388289,468.218151,0,0
1,SGD2,13-01-02,IMP109304,DEC741690,CNTRY653,OFFICE51,2710193100,16784,343358.358598,31334.583459,39097.784585,94337.473500,0,0
2,SGD3,13-01-02,IMP383355,DEC965340,CNTRY680,OFFICE92,6214300000,656,40675.081867,76655.504238,93224.773942,10825.920639,0,0
3,SGD4,13-01-02,IMP512685,DEC244789,CNTRY759,OFFICE59,8703222900,1,24965.116673,4526.575412,5160.500757,758.079630,0,0
4,SGD5,13-01-02,IMP128176,DEC557967,CNTRY528,OFFICE51,3507900000,17655,150957.306315,62715.993325,81152.409190,30518.286546,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,SGD99996,17-12-29,IMP55225,DEC82529,CNTRY277,OFFICE51,8464900000,126,336806.197108,58149.450889,61176.407534,3820.754635,0,0
99996,SGD99997,17-12-29,IMP870848,DEC987092,CNTRY759,OFFICE60,8703232926,1,43.969601,944.709093,1068.969969,925.896409,0,0
99997,SGD99998,17-12-29,IMP244819,DEC384964,CNTRY825,OFFICE45,8481900000,1,32.224977,654.653750,799.860297,408.178992,0,0
99998,SGD99999,17-12-29,IMP403260,DEC208457,CNTRY759,OFFICE60,8703232926,1,198.555892,636.726134,887.735852,61.509931,0,0


In [71]:
samples.to_csv('synthetic-imports-declarations.csv', index=False)